In [ ]:
##### How I think I am going to use the DAQ to time the frequency counter

# connect the Gate In/Out BNC channel to the usb DAQ
# some sort of wizardry to get the DAQ to send a signal to the gate BNC on a particular schedule
# configure the keysight to use an external trigger
# use INIT (NOT READ) to get measurements because it will be faster
# get measurements to output buffer using FETCH

# may want to configure something where I take some number of measurements, off load all the data to the computer, save that to a file, and then repeat
# that way even if something goes wrong I will still get some data on any given run?


In [1]:
#################################################################################################
# THIS IS THE ONE THAT WORKS DO NOT DELETE
#################################################################################################

import nidaqmx
import time
import pyvisa as visa
import numpy as np
import pandas as pd
import Utilities as util
from os import path
import daqUtils
import matplotlib as plt

#do all the document prep at the beginning so it doesn't slow down collection later
#create a new csv file at the specified location
folder = 'Data\\WallTimeExperiment\\'
filename = util.dtStringForFilename()
fp = folder + filename
with open(fp, 'a') as creating_new_csv_file:
    pass
#create an empty data frame and save the headers to the file
df_headers = pd.DataFrame({'Dates':[], 'Frequencies': [], 'Times':[]})
df_headers.to_csv(fp, mode='a', index=False)
print('Output file created')

# open a connection with the keysight
rm = visa.ResourceManager()
# Input the target instrument IP address
Addr = 'USB0::0x0957::0x1807::MY50000728::INSTR'
inst = rm.open_resource(Addr)
inst.encoding = 'latin_1'
inst.source_channel = 'CH1'

trig_count = 5

# reset everything and clear the event queues
inst.write('*RST')
inst.write('STAT:PRES')
inst.write('*CLS')

# set the type of measurement to frequency
inst.write('CONF:FREQ')
inst.write('TRIG:SOUR EXT')
inst.write('TRIG:SLOP POS')
trignum_cmd = 'TRIG:COUNT '+str(trig_count)
inst.write(trignum_cmd)

task = nidaqmx.Task()
task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
print('Starting Collection')
task.start()
task.write(0.0)#make sure we are starting at 0V
inst.write('INIT')

#do one trigger cycle to get rid of the empty data point that apparently gets collected for reasons?
#I hate that this is a thing, but it appears to be a thing, so we're going to roll with it
task.write(2.0)
time.sleep(0.1)
task.write(0.0)
time.sleep(0.1)
inst.query('R?')#remove the empty data point from the data register, so our time stamps will match up with the frequencies collected

how_many_cycles = 2
cycle_num=1

while (cycle_num<=how_many_cycles):
    print('Starting Data Collection Cycle '+str(cycle_num))
    times = [] #the timestamps will go here
    frequencies = ''
    
    times = daqUtils.genAnalogTriggerCycle(task, trig_count, 2.0, 0.0, 0.5, 0.5)

    #task.write(0.0)# always end at 0.0V
    task.stop()
    print('Ending Collection Cycle '+str(cycle_num))

    try:
        frequencies = inst.query('FETC?')
    except: 
        print('I regret to inform you that something done f*cked up and there is no data.')
        print('Ending data collection.')
        inst.close()
        rm.close()
        break


    #now I need to make the data into some sort of format that we can easily put in a text file
    frequencies = util.stringToPandasSeries(frequencies, ',')

    hrdates, hrtimes = util.formatTimestampsForCSV(times)

    times = pd.Series(times)

    df = pd.DataFrame({
        'Date': hrdates,
        'Frequency': frequencies,
        'Time': hrtimes
    })
    df.to_csv(fp, mode='a', index=False, header=False)
    cycle_num = cycle_num+1


#close the connection
openres = rm.list_opened_resources()
print('Closing Connection with ', openres)
inst.close()
rm.close()




Output file created
Starting Collection
Starting Data Collection Cycle 1
Ending Collection Cycle 1
Starting Data Collection Cycle 2
Ending Collection Cycle 2
Closing Connection with  [<'USBInstrument'('USB0::0x0957::0x1807::MY50000728::0::INSTR')>]
